In [19]:
#_*_encoding:utf-8_*_
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests, json
import random
import pandas as pd
import matplotlib.pyplot as plt


## 方法一：使用程式打開瀏覽器

In [6]:
chromeDriver = './chromedriver_mac64/chromedriver'
service_obj = Service(chromeDriver)
driver = webdriver.Chrome(service=service_obj)
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36', }

## 方法二：使用已開啟瀏覽器（需驗證登入時）
終端機輸入 
1. export PATH="/Applications/Google Chrome.app/Contents/MacOS:$PATH"
2. Google\ Chrome --remote-debugging-port=9222 --user-data-dir="~/ChromeProfile"
開啟瀏覽器

In [20]:
chromeDriver = './chromedriver_mac64/chromedriver'
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(executable_path=chromeDriver, options=options)
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36', }

/Users/changchenyu/opt/anaconda3/envs/crawler/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [4]:
# 測試
driver.get("https://interview.tw/auth/login?redirect=https%3A%2F%2Finterview.tw")

## 登入

In [3]:
driver.get("https://interview.tw/auth/login?redirect=https%3A%2F%2Finterview.tw") # 更改網址以前往不同網頁
time.sleep(2) # 等待網頁載入
button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/button[1]")
button.click()
time.sleep(1)
account_text = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
time.sleep(2)
account_text.send_keys('ynmnsysu@gmail.com') # 輸入帳號
login_btn = driver.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button/span')
login_btn.click()
time.sleep(10)
password_text = driver.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
time.sleep(2)
password_text.send_keys('Ynmnsysu2023')  # 輸入密碼
login_btn_2 = driver.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button/span')
login_btn_2.click()
time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/button[1]"}
  (Session info: chrome=112.0.5615.49)
Stacktrace:
0   chromedriver                        0x0000000109e02ae8 chromedriver + 4958952
1   chromedriver                        0x0000000109dfa7b3 chromedriver + 4925363
2   chromedriver                        0x00000001099bc1d6 chromedriver + 475606
3   chromedriver                        0x00000001099ffa20 chromedriver + 752160
4   chromedriver                        0x00000001099ffc81 chromedriver + 752769
5   chromedriver                        0x0000000109a43f64 chromedriver + 1032036
6   chromedriver                        0x0000000109a25c3d chromedriver + 908349
7   chromedriver                        0x0000000109a4125b chromedriver + 1020507
8   chromedriver                        0x0000000109a259e3 chromedriver + 907747
9   chromedriver                        0x00000001099efaef chromedriv

In [21]:
# 設置要爬取的公司代碼, xrbe 後為科技公司
# aQUw:納智捷, xmWa:北智捷, tjzM:中智捷, 0U2N:裕隆汽車, AfWZ:裕隆日產, lwNQ:中華汽車, W139:三陽工業, MGal:福特六和, kchg:國瑞汽車, mU04:台灣本田, i9Um:成運汽車, gWc5:華德動能, 6b50C:鴻華先進, YFgq:為升電裝, xrbe:美光, UDCP:康寧, hKDs:長春石化, 2Iqp:友達光電, l8Yc:捷安特, 2NWa:力積電, 6lU8:華邦電子, U9fO:大立光, hdAT:寶成工業, KYRX:台積電, J7Va:格上租車, NmC3:裕融企業, 2bnH:和泰汽車, cUN9:中華賓士, nYCR:汎德永業, HYto:太古汽車, jT7Q:匯豐汽車
company_code = ['aQUw', 'xmWa', 'tjzM', '0U2N', 'AfWZ', 'lwNQ', 'W139', 'MGal', 'kchg', 'mU04', 'i9Um', 'gWc5', '6b50C', 'YFgq', 'xrbe', 'UDCP', 'hKDs', '2Iqp', 'l8Yc', '2NWa', '6lU8', 'U9fO', 'hdAT', 'KYRX', 'J7Va', 'NmC3', '2bnH', 'cUN9', 'nYCR', 'HYto', 'jT7Q']

In [38]:
# 存放每間公司爬取資訊
df_interview = pd.DataFrame(columns=['company_name', 'vacancies', 'post_time', 'interview_time', 'rank', 'status', 'difficult', 'process', 'question', 'suggestion'])

# 不同公司
for c in company_code:
    # 不同頁面
    for page_num in range(1, 1000):
        # 設置網頁網址
        url = f'https://interview.tw/c/{c}?page={page_num}&sort=newest'
        # url = f'https://interview.tw/c/0U2N?page=1&sort=newest'
        driver.get(url)
        # 獲取網頁內容
        pageSource = driver.page_source
        soup = BeautifulSoup(pageSource, 'html.parser')
        # 公司名稱
        _company_name = soup.find('h1', {'class': 'fz-h2 fz-tit'}).text
        # 取得所以有面試文章區塊
        content = soup.find_all('div', {'class':'iw_experience-item'})
        if (len(content)!=0):
        # 跟去每個面試文章取得職業名稱
            for i in content:
                # 職缺
                _vacancies = i.find('a', {'class':'fz-s1 fz-tit'}).text
                # 分享時間
                _post_time = i.find_all('span', {'class':'fz-s4 cl-basic-500'})[1].text[:-3]
                # 面試時間
                _interview_time = i.find_all('span', {'class':'fz-s2'})[1].text
                # 評價（幾顆星）
                _rank = len(i.find_all('li', {'class':'iw_rate'}))
                # 面試狀態
                _status = i.find_all('span', {'class':'fz-s2'})[2].text
                # 難易度
                _difficult = i.find_all('span', {'class':'fz-s2'})[3].text
                # 面試過程
                try:
                    _process_div = i.find_all('div', {'class':'iw_info-wrap__con cl-basic-700'})
                    _process = _process_div[1].find('span').text
                    # _process = i.find_all('span', {'class':'fz-t3'})[0].text
                except:
                    print("面試過程")
                    print(_company_name)
                    print("page: ",page_num)
                    print(_vacancies)
                    _process = None
                # 面試問題
                try:
                    _question = []
                    _Q = []
                    _A = []
                    ans_title = i.find_all('div', {'class':'ans-title'})
                    for q in ans_title:
                        _Q.append(q.find('span').text)
                    ans_content = i.find_all('div', {'class':'ans-content'})
                    for q in ans_content:
                        _A.append(q.find('span').text)
                    for k in range(0, len(_Q)):
                        tmp = {
                            "Q": _Q[k],
                            "A": _A[k]
                        }
                        _question.append(tmp)
                except:
                    print("面試問題")
                    print(_company_name)
                    print("page: ",page_num)
                    print(_vacancies)
                    _question = None
                # 面試建議
                try:
                    _suggestion_div = i.find_all('div', {'class':'iw_info-wrap__con cl-basic-700'})
                    _suggestion = _suggestion_div[2].find('span').text
                    # _suggestion = i.find_all('span', {'class':'fz-t3'})[1].text
                except Exception as e:
                    _suggestion = None
                tmp_df_interview = pd.DataFrame({
                    "company_name": [_company_name],
                    "vacancies": [_vacancies],
                    "post_time": [_post_time],
                    "interview_time": [_interview_time],
                    "rank": [_rank],
                    "status": [_status],
                    "difficult": [_difficult],
                    "process": [_process],
                    "question": [_question],
                    "suggestion": [_suggestion]
                }, index=[len(df_interview)])
                df_interview = df_interview.append(tmp_df_interview)
        else:
            break
        time.sleep(random.uniform(1.0, 2.0)) 
    # 將爬取到的信息保存到 MySQL 資料庫中
#     sql = "INSERT INTO your_table (title, link, author, date, content) VALUES (%s, %s, %s, %s, %s)"
#     # cursor.execute(sql, (title, link, author, date, content))
#     # db.commit()

面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
企劃管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
行銷專員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
人力資源管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
秘書
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
人資
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
資深人資管理師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
行銷
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
業務
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
行銷
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
材質規劃師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
DMS-工程師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
銷售顧問
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
財務人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  1
技師
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
業務
面試過程
裕隆集團_納智捷汽車股份有限公司
page:  2
行政人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
行政人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
服務專員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
行政專員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
企劃人員
面試建議
裕隆集團_納智捷汽車股份有限公司
page:  2
人資專員
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
行政人員
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
企劃專員
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
汽車銷售顧問
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
業務
面試建議
Luxgen_北智捷汽車股份有限公司
page:  1
業務
面試建議
Luxgen_北智捷汽車股份有限公司


In [40]:
df_interview

company_name vacancies   post_time interview_time rank status  \
0     裕隆集團_納智捷汽車股份有限公司     企劃管理師  2022.08.24        2022.07    3    未錄取   
1     裕隆集團_納智捷汽車股份有限公司       管理師  2022.02.21        2019.02    1    無聲卡   
2     裕隆集團_納智捷汽車股份有限公司      行銷專員  2022.01.13        2017.09    3     錄取   
3     裕隆集團_納智捷汽車股份有限公司   人力資源管理師  2021.09.12        2021.09    5    未錄取   
4     裕隆集團_納智捷汽車股份有限公司        秘書  2021.04.21        2017.09    5     錄取   
...                ...       ...         ...            ...  ...    ...   
7608        匯豐汽車股份有限公司    產品企劃專員  2018.04.20        2017.12    4     錄取   
7609        匯豐汽車股份有限公司   SUM服務專員  2018.03.23        2010.06    5     錄取   
7610        匯豐汽車股份有限公司   總公司-品牌部  2018.01.24        2016.06    3     錄取   
7611        匯豐汽車股份有限公司        技師  2017.12.24        2002.06    4     錄取   
7612        匯豐汽車股份有限公司       貸款課  2017.12.24        2017.11    3     錄取   

     difficult        process  \
0           普通  針對專業提出疑問，並...   
1            低  整體面試過程約2小時...   
2           普通  第一次面試先簡單自我...   
3           超高  第一關，人資面試\n...   
4           普通  在台南東區的廠二樓面...   
...        ...            ...   
7608        簡單     簡報說明令人印象深刻   
7609        簡單  面試過程其實很好，而...   
7610        簡單  帶到會議室，等待面試...   
7611        簡單  直接面試 問維修相關...   
7612        簡單  面試的過程談論一些公...   

                                               question suggestion  
0     [{'Q': '這個品牌業界傳的都不太好，為什麼願意來納智捷面試', 'A': '因為我本來...       None  
1             [{'Q': '對於納智捷的了解', 'A': '依照職位跟公司官網做...'}]       None  
2                [{'Q': '會不會開車', 'A': '有駕照，但當時還不熟...'}]       None  
3                       [{'Q': '結算薪資怎麼做', 'A': '照実回答'}]       None  
4            [{'Q': '與人交談方面的問題', 'A': '我是個開朗的人，比較...'}]       None  
...                                                 ...        ...  
7608          [{'Q': '針對產品提出看法', 'A': '直接腦袋中的想法講出...'}]       None  
7609  [{'Q': '時間有點久不太記得，但是要看你遇到的廠長是好是壞!', 'A': '時間有點...       None  
7610  [{'Q': '對於行銷與品牌的各種看法，EXCEL操作技能，網站經營(GA)，網路行銷規劃...       None  
7611     [{'Q': '需配合值班可否接受 前份工作為何離職', 'A': '可以

職涯規劃'}]       None  
7612         [{'Q': '是否能接受假日工作', 'A': '不需要特別建議，採個...'}]       None  

[7613 rows x 10 columns]

In [41]:
df_interview.to_csv('interview_test.csv', encoding='utf-8')